In [12]:
# create and run the task
images_path = os.path.join(os.getcwd(), 'images')
ZIP_ARCHIVE = 'http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip'
TAR_ARCHIVE = 'https://fpsignals-public.s3.amazonaws.com/catsndogs.tar.gz'

## cleanup

In [20]:
import shutil
shutil.rmtree(os.path.join(os.getcwd(), 'images'))

In [21]:
!mlrun clean -p -r

[mlrun] 2020-03-13 10:58:00,329 using in-cluster config.
state      started          type     name


In [14]:
open_archive_task = mlrun.NewTask(
    'download-tar',
    params={'key'        : 'tar-data', 'archive_url': TAR_ARCHIVE},
    artifact_path=images_path)

# run
run2 = fn.run(open_archive_task)

[mlrun] 2020-03-13 11:01:58,380 starting run download-tar uid=ba62f7dff43045e899d1c7f83692e030  -> http://mlrun-api:8080
[mlrun] 2020-03-13 11:01:58,463 Job is running in the background, pod: download-tar-gv69v
[mlrun] 2020-03-13 11:02:03,461 Verified directories
[mlrun] 2020-03-13 11:02:03,461 opening tar_gz
[mlrun] 2020-03-13 11:02:20,178 log artifact tar-data at /User/functions-refac/open_archive/images/User/functions-refac/open_archive/images, size: None, db: Y

[mlrun] 2020-03-13 11:02:20,191 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...92e030,0,Mar 13 11:02:03,completed,download-tar,host=download-tar-gv69vkind=jobowner=admin,,archive_url=https://fpsignals-public.s3.amazonaws.com/catsndogs.tar.gzkey=tar-data,,tar-data


to track results use .show() or .logs() or in CLI: 
!mlrun get run ba62f7dff43045e899d1c7f83692e030  , !mlrun logs ba62f7dff43045e899d1c7f83692e030 
[mlrun] 2020-03-13 11:02:27,658 run executed, status=completed


In [13]:
open_archive_task = mlrun.NewTask(
    'download-zip',
    params={'key'        : 'images'},
    inputs={'archive_url': ZIP_ARCHIVE},
    artifact_path=images_path)

# run
run1 = fn.run(open_archive_task)

[mlrun] 2020-03-13 11:01:40,058 starting run download-zip uid=512ebe4799774dd5b7d03f1720104d2b  -> http://mlrun-api:8080
[mlrun] 2020-03-13 11:01:40,146 Job is running in the background, pod: download-zip-gn6qb
[mlrun] 2020-03-13 11:01:45,085 downloading http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip to local tmp
[mlrun] 2020-03-13 11:01:46,756 Verified directories
[mlrun] 2020-03-13 11:01:46,756 opening zip
[mlrun] 2020-03-13 11:01:55,023 log artifact images at /User/functions-refac/open_archive/images/User/functions-refac/open_archive/images, size: None, db: Y

[mlrun] 2020-03-13 11:01:55,034 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...104d2b,0,Mar 13 11:01:45,completed,download-zip,host=download-zip-gn6qbkind=jobowner=admin,archive_url,key=images,,images


to track results use .show() or .logs() or in CLI: 
!mlrun get run 512ebe4799774dd5b7d03f1720104d2b  , !mlrun logs 512ebe4799774dd5b7d03f1720104d2b 
[mlrun] 2020-03-13 11:01:58,362 run executed, status=completed


In [13]:
open_archive_task = mlrun.NewTask(
    'download-zip',
    params={'key'        : 'images'},
    inputs={'archive_url': ZIP_ARCHIVE},
    artifact_path=images_path)

# run
run1 = fn.run(open_archive_task)

[mlrun] 2020-03-13 11:01:40,058 starting run download-zip uid=512ebe4799774dd5b7d03f1720104d2b  -> http://mlrun-api:8080
[mlrun] 2020-03-13 11:01:40,146 Job is running in the background, pod: download-zip-gn6qb
[mlrun] 2020-03-13 11:01:45,085 downloading http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip to local tmp
[mlrun] 2020-03-13 11:01:46,756 Verified directories
[mlrun] 2020-03-13 11:01:46,756 opening zip
[mlrun] 2020-03-13 11:01:55,023 log artifact images at /User/functions-refac/open_archive/images/User/functions-refac/open_archive/images, size: None, db: Y

[mlrun] 2020-03-13 11:01:55,034 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...104d2b,0,Mar 13 11:01:45,completed,download-zip,host=download-zip-gn6qbkind=jobowner=admin,archive_url,key=images,,images


to track results use .show() or .logs() or in CLI: 
!mlrun get run 512ebe4799774dd5b7d03f1720104d2b  , !mlrun logs 512ebe4799774dd5b7d03f1720104d2b 
[mlrun] 2020-03-13 11:01:58,362 run executed, status=completed


In [4]:
FUNCTION = 'load_dataset'

task_params = {
    "name"           : "load_data", 
    "params"         : {
        "dataset"     : 'iris'
    },
    "artifact_path"  : '/User/functions/load_dataset/artifacts'
}

In [9]:
import mlrun, os
mlrun.mlconf.dbpath = 'http://mlrun-api:8080'
mlrun.mlconf.hub_url = 'https://raw.githubusercontent.com/yjb-ds/functions/init_fixes/{name}/function.yaml'

urls = ['function.yaml', f'db://{FUNCTION}:latest', f'hub://{FUNCTION}']

tasks = []
for url in urls:
    try:
        rfn = mlrun.import_function(url).apply(mlrun.mount_v3io())
        tsk = rfn.run(mlrun.NewTask(**task_params)
        tasks.append(tsk)
    except Exception as e:
        print(url, str(e))
        tasks.append(f'error {url}, {e}')
    
    print("++++++++++++++++++++++++++++++++++++++++++++++++")